In [103]:
import pandas as pd
import locale
from datetime import datetime, timedelta
import math
import numpy as np
from numba import jit

locale.setlocale(locale.LC_ALL, 'EN_US')



def floor_ft(dt, delta):
    return datetime.min + math.floor((dt - datetime.min) / delta) * delta

def find_dispatch(price, bid, plant_size, dispatchPrice):
    df.loc[(price > bid), dispatchPrice] = plant_size
    df.loc[(price <= bid), dispatchPrice] = 0

def find_price(price, bid, dispatchPrice):
    df.loc[(price > bid), dispatchPrice] = price
    df.loc[(price <= bid), dispatchPrice] = 0

def profit(dispatch, price, bid, dispatchPrice):
    df.loc[(dispatch > 0), dispatchPrice] = price - (bid * dispatch * (5/60))
    df.loc[(dispatch <= 0), dispatchPrice] = 0

def thirty_min_prices(dic, flooredTimes, col):
    df.loc[(dic[flooredTimes] == flooredTimes), col] = dic[flooredTimes]

def power_station_model(future_price, rrp, power_station_dispatch, bid, ramp):
    for i in range(1, len(rrp)):
        if (future_price[i] > bid) or (rrp[i] > 300):
            re = power_station_dispatch[i - 1] + ramp
            if re > 1: re = 1
        else:
            re = power_station_dispatch[i - 1]- ramp
            if re < 0: re = 0
        power_station_dispatch[i] = re
    return power_station_dispatch

def floor_time(time, col):
    dfALL.loc[col]  =floor_ft(time, timedelta(minutes=30))

In [104]:
dfALL = pd.read_csv("./AEMOData/2019data.csv")
dfALL['SETTLEMENTDATE'] = pd.to_datetime(dfALL['SETTLEMENTDATE'])
dfALL['SETTLEMENTDATE'] = dfALL["SETTLEMENTDATE"].apply(pd.Timestamp.to_pydatetime)
# floor_time(dfALL['SETTLEMENTDATE'], 'flooredtimes')

# dfALL['flooredtimes'] = dfALL['SETTLEMENTDATE'].apply(lambda row: floor_ft(row.to_pydatetime(), timedelta(minutes=30)))
dfALL['flooredtimes'] = dfALL['SETTLEMENTDATE'].dt.floor('30T')
# state = 'SA1'
#
# df = dfALL[dfALL['REGIONID'] == state]
# thirtymindf = df.resample('30T').mean()
# thirtymindf = thirtymindf.reset_index()


In [105]:
bid = 300
plant_size = 1

bid = int(input('Pick a bid: '))
state = input('Pick a state {NSW1, QLD1, SA1, VIC1}: ')

df = dfALL[dfALL['REGIONID'] == state]

df = df.set_index('SETTLEMENTDATE')
df = df.loc[~df.index.duplicated(keep='first')]
df.index = pd.to_datetime(df.index)

thirtymindf = df.resample('30T').mean()
thirtymindf.index = pd.to_datetime(thirtymindf.index)
dic = thirtymindf.to_dict('index')
# thirtymindf = thirtymindf.reset_index()
df = df.reset_index()


df['30minprices'] =  df['flooredtimes'].map(dic)
df['30minprices'] = pd.json_normalize(df['30minprices'])
# pd.DataFrame(df['30minprices'].tolist())
# df['30minprices'] = df['flooredtimes'].apply(lambda time: thirtymindf['RRP'][(thirtymindf['SETTLEMENTDATE'] == time)].values[0])



find_dispatch(df['RRP'], bid, plant_size, 'Dispatch')
find_price(df['RRP'], bid, '5MinPrice')
df['Revenue'] = df['5MinPrice'] * df['Dispatch'] * (5/60)
profit(df['Dispatch'], df['Revenue'], bid, 'Profit')

fiveminrevenue = df['Revenue'].sum()
fiveminprofit = df['Profit'].sum()
print('State: '+state)
print('Fuel Cost: '+str(bid))
print('Perfectly responsive plant: ' + str(df['Dispatch'].sum()) + 'MWh')

print('5 Minute Revenue = ' + locale.currency(df['Revenue'].sum(), grouping=True))
print('5 Minute Profic = ' + locale.currency(df['Profit'].sum(), grouping=True))

df['30minRevenue'] = df['30minprices'] * df['Dispatch'] * (5/60)
profit(df['Dispatch'], df['30minRevenue'], bid, '30minProfit')

print('30 Minute Revenue = ' + locale.currency(df['30minRevenue'].sum(), grouping=True))
print('30 Minute Profit = ' + locale.currency(df['30minProfit'].sum(), grouping=True))

thirtyminrevenue = df['30minRevenue'].sum()
thirtyminprofit = df['30minProfit'].sum()

revenue_increase = (fiveminrevenue / thirtyminrevenue) - 1
profit_increase = (fiveminprofit / thirtyminprofit) - 1

print("Revenue increase: " +"{:.2%}".format(revenue_increase))
print("Profit increase: " +"{:.2%}".format(profit_increase))



State: NSW1
Fuel Cost: 300
Perfectly responsive plant: 178.0MWh
5 Minute Revenue = $10,833.01
5 Minute Profic = $6,383.01
30 Minute Revenue = $6,227.73
30 Minute Profit = $1,777.73
Revenue increase: 73.95%
Profit increase: 259.05%


In [106]:


ramp = 0.125

bid = int(input('Pick a bid: '))
state = input('Pick a state {NSW1, QLD1, SA1, VIC1}: ')

df = dfALL[dfALL['REGIONID'] == state]

df = df.set_index('SETTLEMENTDATE')
df = df.loc[~df.index.duplicated(keep='first')]
df.index = pd.to_datetime(df.index)

thirtymindf = df.resample('30T').mean()
thirtymindf.index = pd.to_datetime(thirtymindf.index)
dic = thirtymindf.to_dict('index')
# thirtymindf = thirtymindf.reset_index()
df = df.reset_index()


df['30minprices'] =  df['flooredtimes'].map(dic)
df['30minprices'] = pd.json_normalize(df['30minprices'])

df.loc[0, 'power_station_dispatch'] = 0
df['futurePrice'] = df['RRP'].shift(-2)

psm = power_station_model(df.futurePrice.values.tolist(), df.RRP.values.tolist(), df.power_station_dispatch.values.tolist(), bid, ramp)

df['power_station_dispatch'] = pd.Series(psm)

# print("done")

df['5minRevenuePS'] = df['RRP'] * df['power_station_dispatch'] * (5/60)
profit(df['power_station_dispatch'], df['5minRevenuePS'], bid, '5minProfitPS')

fiveminrevenuePS = df['5minRevenuePS'].sum()
fiveminprofitPS = df['5minProfitPS'].sum()
print('State: '+state)
print('Fuel Cost: '+str(bid))
print('Power Statione with '+str(ramp)+' ramp: '+str(df['power_station_dispatch'].sum())+'MW')

print('5 Minute Revenue = '+locale.currency(df['5minRevenuePS'].sum(), grouping=True))
print('5 Minute Profit = '+locale.currency(df['5minProfitPS'].sum(), grouping=True))

df['30minRevenuePS'] = df['30minprices'] * df['power_station_dispatch'] * (5/60)
profit(df['power_station_dispatch'], df['30minRevenuePS'], bid, '30minProfitPS')

print('30 Minute Revenue = '+locale.currency(df['30minRevenuePS'].sum(), grouping=True))
print('30 Minute Profit = '+locale.currency(df['30minProfitPS'].sum(), grouping=True))

thirtyminrevenuePS = df['30minRevenuePS'].sum()
thirtyminprofitPS = df['30minProfitPS'].sum()

revenue_increasePS = (fiveminrevenuePS / thirtyminrevenuePS) - 1
profit_increasePS = (fiveminprofitPS / thirtyminprofitPS) - 1

print("Revenue increase: " +"{:.2%}".format(revenue_increasePS))
print("Profit increase: " +"{:.2%}".format(profit_increasePS))

State: SA1
Fuel Cost: 150
Power Statione with 0.125 ramp: 6195.75MW
5 Minute Revenue = $243,463.10
5 Minute Profit = $166,016.23
30 Minute Revenue = $239,473.64
30 Minute Profit = $162,026.76
Revenue increase: 1.67%
Profit increase: 2.46%
